In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
"""
本地使用全部原始数据
"""

is_sample_data = True
islocal = True

In [3]:
if islocal:
    %time df_app = pd.read_csv('wsdm_train_data/app_launch_logs.csv')
    %time df_inter = pd.read_csv('wsdm_train_data/user_interaction_data.csv')
    %time df_play = pd.read_feather('wsdm_train_data/user_playback_data.feather')
    %time df_port = pd.read_csv('wsdm_train_data/user_portrait_data.csv')
    %time df_video = pd.read_csv('wsdm_train_data/video_related_data.csv')
    test = pd.read_csv('test_B数据集.csv')
elif is_sample_data:
    df_app = pd.read_csv('/kaggle/input/iqiyi/df_app')
    df_inter = pd.read_csv('/kaggle/input/iqiyi/df_inter')
    df_play = pd.read_csv('/kaggle/input/iqiyi/df_play')
    df_port = pd.read_csv('/kaggle/input/iqiyi/df_port')
    df_video = pd.read_csv('/kaggle/input/iqiyi/df_video')
    test = pd.read_csv('/kaggle/input/aiqiyi-competition/test-a.csv')
else:
    df_app = pd.read_csv('/kaggle/input/aiqiyi-competition/app_launch_logs.csv')
    df_inter = pd.read_csv('/kaggle/input/aiqiyi-competition/user_interaction_data.csv')
    df_play = pd.read_csv('/kaggle/input/aiqiyi-competition/user_playback_data.csv')
    df_port = pd.read_csv('/kaggle/input/aiqiyi-competition/user_portrait_data.csv')
    df_video = pd.read_csv('/kaggle/input/aiqiyi-competition/video_related_data.csv')

    test = pd.read_csv('/kaggle/input/aiqiyi-competition/test-a.csv')

Wall time: 1.22 s
Wall time: 50.4 ms
Wall time: 2.5 s
Wall time: 271 ms
Wall time: 2.98 s


In [4]:
# df_app = df_app[df_app.date >= df_app[['user_id','date']].groupby('user_id')['date'].transform(
# 'max') - 60]
# len(df_app)

In [6]:
# np.random.seed(0)
# def f(x,ratio=0.05):
#     xmax, x_min = x.max(),x.min()
#     n_sample = min(xmax - x_min - 6,np.ceil((xmax - x_min)*ratio))
#     n_sample = int(n_sample) 
#     if n_sample > 0:
#         return np.random.choice(np.arange(x_min,xmax-6), size=n_sample, replace=False).tolist()
#     else:
#         return []
# train_date = df_app.groupby('user_id')[['date']].agg(f)
# train_date = train_date[train_date.date.apply(len)>0]
# train_date = train_date.explode('date').reset_index()

In [7]:
"""
训练集抽样
"""

np.random.seed(0)
def f(x):
    xmax, x_min = x.max(),x.min()
    if xmax - x_min > 10:
        return np.random.randint(low=x_min,high=xmax-6)
    else:
        return np.nan
train_date = df_app.groupby('user_id')['date'].agg(f)
train_date = train_date.reset_index()
train_date = train_date[~train_date.date.isnull()]

In [8]:
train_date['date'] = train_date['date'].astype('int')
df = pd.concat([df_app,train_date,test.rename(columns={'end_date':'date'})],ignore_index=True)

In [9]:
df = df.groupby(['user_id','date']).agg({'launch_type':['count','sum']}).reset_index()
df.columns =['user_id','date','launch_count','type1_count']

In [10]:
for col in ['launch_count','type1_count']:
    df[col] = df[col].fillna(0).astype('int8')
df['type0_count'] = df['launch_count'] - df['type1_count']
df['islogin'] = (df['launch_count'] > 0).astype('int8')

In [11]:
"""
生产label
"""

import numba

@numba.jit(nopython=True)
def get_ndays_login(arr,n_days):
    res = np.zeros(len(arr),dtype=np.int8)
    for i in numba.prange(len(res)):
        for j in range(1,n_days+1):
            if i+j < len(arr) and arr[i+j][0] == arr[i][0] and arr[i+j][1] - arr[i][1] <= n_days:
                res[i] += arr[i+j][2]
            else:
                break
    return res

%time df['label'] = get_ndays_login(df[['user_id','date','islogin']].values,n_days=7)
df['label'] = np.where(df['date'] >= df['user_id'].map(df.groupby('user_id')['date'].max())-6,-1,df.label)


Wall time: 1.4 s


生成特征

In [12]:
@numba.jit(nopython=True)
def future_n_login_mean(arr,n_days):
    res = np.zeros(len(arr),dtype=np.float32)
    res -= 1
    tmp_id = arr[0][0]
    for i in range(1,len(arr)):
        if arr[i][0] != tmp_id:
            tmp_id =  arr[i][0]
            continue
        tmp_sum = 0
        count = 0
        for j in range(1,i+1):
            if arr[i-j][0] == tmp_id:
                if arr[i][1] - arr[i-j][1] >= n_days and arr[i-j][2] >= 0:
                    tmp_sum += arr[i-j][2]
                    count += 1
            else:

                break
        if count > 0:
            res[i] = tmp_sum / count
    return res
    

In [13]:
for n_days in range(1,7):
    df['tmp'] = get_ndays_login(df[['user_id','date','islogin']].values,n_days)
    df['tmp'] = np.where(df['date'] >= df['user_id'].map(df.groupby('user_id')['date'].max())-n_days+1,-1,df.tmp)
    df[f'future_{n_days}_login_mean'] = future_n_login_mean(df[['user_id','date','tmp']].values,n_days)
    df[f'future_{n_days}_login_mean'] = np.where(df[f'future_{n_days}_login_mean'] <0,np.nan,df[f'future_{n_days}_login_mean'] )
df[f'future_label_login_mean'] = future_n_login_mean(df[['user_id','date','label']].values,7)
df[f'future_label_login_mean'] = np.where(df[f'future_label_login_mean'] <0,np.nan,df[f'future_label_login_mean'] )

del df['tmp']

In [14]:
# @numba.jit(nopython=True)
# def is_ndays_login(x,ndays):
#     res = np.zeros(len(x),dtype=np.int8)
#     #res -= 1
#     login_day = np.zeros(300,dtype=np.int8)
#     for i in range(len(x)):
#         login_day[x[i]] = 1
#         if login_day[x[i]-ndays] == 1:
#             res[i] = 1
#         else:
#             res[i] = 0
#     return res
# for ndays in range(1,8):
#     df[f'is_{ndays}_login_01'] = df.groupby('user_id')['date'].transform(lambda x:is_ndays_login(x.values,ndays))

In [15]:
@numba.jit(nopython=True)
def max_login_day(arr):
    res = np.zeros(len(arr),dtype=np.int8)
    res -= 1
    tmp_id = -1
    for i in range(0,len(arr)):
        if arr[i][0] != tmp_id:
            tmp_id = arr[i][0]
            tmp_day = 0
            continue
        else:
            if arr[i][1] - arr[i-1][1] == 1 and arr[i-1][2] == 1:
                tmp_day += 1
            else:
                tmp_day = 0

        res[i] = tmp_day
    return res
    

df['max_login_day'] = max_login_day(df[['user_id','date','islogin']].values)
df['max_login_day'] = np.where(df['max_login_day'] < 0,0,df['max_login_day'])


In [16]:
@numba.jit(nopython=True)
def last_n_login_day(arr,n):
    res = np.zeros(len(arr),dtype=np.int8)
    res -= 1
    for i in range(0,len(arr)):
        count = n
        for j in range(1,130):
            if i - j >= 0 and arr[i-j][0] == arr[i][0]:
                count -= arr[i-j][2]
                if count == 0:
                    res[i] = arr[i][1] - arr[i-j][1]
                    break
            else:
                break
    return res
for n in range(1,15):
    df[f'last{n}_login'] = last_n_login_day(df[['user_id','date','islogin']].values,n)
    #df[f'last{n}_login'] = df[f'last{n}_login'].replace(-1,np.nan)
    if n == 1:
        df[f'last{n}_login'] = np.where(df[f'last{n}_login'] < 0,7,df[f'last{n}_login'])
    else:
        df[f'last{n}_login'] = np.where(df[f'last{n}_login'] < 0,df[f'last{n-1}_login']*(n/(n-1)),df[f'last{n}_login'])
    

In [17]:
df['day_minus_min_date'] = df['date'] -  df[['user_id','date']].groupby('user_id')['date'].transform('min')
df['cum_login'] = df[['user_id','islogin']].groupby('user_id')['islogin'].cumsum()
df['cum_launch_count'] = df[['user_id','launch_count']].groupby('user_id')['launch_count'].cumsum()
df['cum_type1_count_count'] = df[['user_id','type1_count']].groupby('user_id')['type1_count'].cumsum()
df['cum_lauch_ratio'] = df['cum_type1_count_count'] / df['cum_launch_count']
df['cum_login_mean'] = df['cum_login'] / df['day_minus_min_date']
df['cum_launch_mean'] = df['cum_launch_count'] / df['day_minus_min_date']
# df['cum_max_login_day_mean'] = df[['user_id','max_login_day']].groupby('user_id').cummean()

In [18]:
%%time 

df_inter_pivot = pd.pivot_table(index=['user_id','date'],data=df_inter,columns=['interact_type'],aggfunc=['count'],fill_value=0).reset_index()
df_inter_pivot.columns = ['user_id','date'] + [f'inter_{i}_count' for i in range(1,12)]
df_inter = df_inter.groupby(['user_id','date']).agg(
    {'item_id':['count','nunique'],
    'interact_type':['nunique']}
)
df_inter = df_inter.reset_index()
df_inter.columns = ['user_id','date','inter_count','inter_item_nunique','inter_type_nunique']
df_inter['inter_ratio'] = df_inter['inter_count'] / df_inter['inter_item_nunique']

Wall time: 176 ms


In [19]:
df_inter_pivot['inter_argmax'] = np.argmax(df_inter_pivot[[f'inter_{i}_count' for i in range(1,12)]].values,axis=1)

In [20]:
df_video['father_id_count'] = df_video.groupby('father_id')['item_id'].transform('nunique')

#df_play = pd.read_csv('/kaggle/input/iqiyi/df_play')
df_play = df_play.merge(df_video,on = 'item_id',how='left')

In [21]:
df_play.head()

,user_id,item_id,playtime,date,duration,father_id,tag_list,cast,father_id_count
0,10057286,20628283.0,2208.612,145,11.0,30038149.0,50444644;50260120;50057164;50323629;50107634;5...,40015213;40019805;40021027;40005903;40004251;4...,1317.0
1,10522615,23930557.0,31.054,145,4.0,NaN,NaN,NaN,NaN
2,10494028,20173699.0,115.952,145,6.0,NaN,50207929;50391748;50335664,NaN,NaN
3,10181987,21350426.0,1.585,145,NaN,NaN,NaN,NaN,NaN
4,10439175,22946929.0,51.726,145,8.0,30094115.0,50205544;50392322;50150172;50050918;50237137;5...,NaN,204.0


In [22]:
df_play['duration'] = np.where(df_play['duration']>15,15,df_play['duration'])
#df_play['duration'] = df_play['duration'].astype('int8')
df_play_pivot = pd.pivot_table(index=['user_id','date'],data=df_play[['user_id','date','duration','item_id']],columns=['duration'],aggfunc=['count'],fill_value=0).reset_index()
df_play_pivot.columns = ['user_id','date'] + [f'duration{i}' for i in range(1,16)]
df_play_pivot['duration_argmax'] = np.argmax(df_play_pivot[[f'duration{i}' for i in range(1,16)]].values,axis=1)
#df_play= df_play.merge(df_tmp, on =['user_id','date'])

In [23]:
df_play_pivot

,user_id,date,duration1,duration2,duration3,duration4,duration5,duration6,duration7,duration8,duration9,duration10,duration11,duration12,duration13,duration14,duration15,duration_argmax
0,10000000,131,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,10
1,10000000,132,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,10
2,10000000,195,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,10
3,10000002,128,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,11
4,10000002,140,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4548487,10599997,161,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,10
4548488,10599997,164,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,10
4548489,10599997,165,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,10
4548490,10599997,166,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,10


In [24]:
df_play  = df_play.merge(df[df.label>=0][['user_id','date','label']],on = ['user_id','date'], how='left')

In [25]:
df_play.head()

,user_id,item_id,playtime,date,duration,father_id,tag_list,cast,father_id_count,label
0,10057286,20628283.0,2208.612,145,11.0,30038149.0,50444644;50260120;50057164;50323629;50107634;5...,40015213;40019805;40021027;40005903;40004251;4...,1317.0,1.0
1,10522615,23930557.0,31.054,145,4.0,NaN,NaN,NaN,NaN,2.0
2,10494028,20173699.0,115.952,145,6.0,NaN,50207929;50391748;50335664,NaN,NaN,1.0
3,10181987,21350426.0,1.585,145,NaN,NaN,NaN,NaN,NaN,7.0
4,10439175,22946929.0,51.726,145,8.0,30094115.0,50205544;50392322;50150172;50050918;50237137;5...,NaN,204.0,2.0


In [26]:
"""
参考baseline
"""

def target_encoding(name, df, m=1):
#     df[name] = df[name].str.split(";")
   # df = df.explode(name)
    overall = df[~df.label.isnull()]["label"].mean()
    df = df.groupby(name).agg(
        freq=("label", "count"), 
        in_category=("label", np.mean)
    ).reset_index()
    df["weight"] = df["freq"] / (df["freq"] + m)
    df["score"] = df["weight"] * df["in_category"] + (1 - df["weight"]) * overall
    return df

In [27]:
father_id_score = target_encoding("father_id", df_play[df_play.label>=0])
father_id_score

,father_id,freq,in_category,weight,score
0,30000000.0,7,5.857143,0.875000,5.721450
1,30000001.0,2,4.500000,0.666667,4.590533
2,30000002.0,17,3.588235,0.944444,3.653978
3,30000003.0,1,7.000000,0.500000,5.885799
4,30000004.0,2,4.000000,0.666667,4.257199
...,...,...,...,...,...
84375,30094643.0,106,3.962264,0.990654,3.969828
84376,30094644.0,5,4.400000,0.833333,4.461933
84377,30094645.0,47,5.680851,0.979167,5.661908
84378,30094646.0,2,3.500000,0.666667,3.923866


In [28]:
item_id_score = target_encoding("item_id", df_play[df_play.label>=0])
item_id_score

,item_id,freq,in_category,weight,score
0,20000001.0,2,7.000000,0.666667,6.257199
1,20000002.0,3,3.333333,0.750000,3.692899
2,20000003.0,1,0.000000,0.500000,2.385799
3,20000004.0,15,5.266667,0.937500,5.235725
4,20000005.0,1,7.000000,0.500000,5.885799
...,...,...,...,...,...
4625291,25074434.0,21,5.428571,0.954545,5.398709
4625292,25074435.0,2,3.500000,0.666667,3.923866
4625293,25074436.0,1,5.000000,0.500000,4.885799
4625294,25074437.0,23,3.956522,0.958333,3.990483


In [29]:
date_score = target_encoding("date", df_play[df_play.label>=0])
date_score

,date,freq,in_category,weight,score
0,100,304859,4.224986,0.999997,4.224988
1,101,303845,4.189511,0.999997,4.189513
2,102,317462,4.081225,0.999997,4.081228
3,103,311854,3.930692,0.999997,3.930695
4,104,283605,3.997609,0.999996,3.997612
...,...,...,...,...,...
111,211,234607,5.308772,0.999996,5.308769
112,212,231197,5.400170,0.999996,5.400167
113,213,214714,5.471944,0.999995,5.471941
114,214,190880,5.801734,0.999995,5.801729


In [30]:
import gc
def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df

In [31]:
df_play = reduce_mem(df_play)
df_video = reduce_mem(df_video)
df = reduce_mem(df)
df_port = reduce_mem(df_port)
df_inter = reduce_mem(df_inter)

6292.36 Mb, 3432.19 Mb (45.45 %)
194.55 Mb, 113.49 Mb (41.67 %)
1669.53 Mb, 644.08 Mb (61.42 %)
40.99 Mb, 19.35 Mb (52.78 %)
1.80 Mb, 0.49 Mb (72.91 %)


In [32]:
# father_id_score = reduce_mem(father_id_score)

In [33]:
df_play = df_play.merge(
    father_id_score[['father_id','freq','score']].rename(columns ={
        'freq':'father_id_freq','score':'father_id_score'
    }),
    on = 'father_id',
    how='left'
)

In [34]:
df_play = df_play.merge(
    item_id_score[['item_id','freq','score']].rename(columns = {
        'freq':'item_id_freq','score':'item_id_score'
    }),
    on = 'item_id',
    how='left'
)

In [35]:
df_play['duration_mean'] = df_play['duration'].map(
{1.0: 3.7708806818181815,
 2.0: 6.50662590535973,
 3.0: 11.386861075884855,
 4.0: 19.458476060964017,
 5.0: 31.75858442584106,
 6.0: 53.154826728126096,
 7.0: 92.99056310706094,
 8.0: 151.86914162681802,
 9.0: 266.6738094823002,
 10.0: 534.0026326330363,
 11.0: 1297.075341192911,
 12.0:  2048,
 13.0: 3072,
 14.0: 4096,
 15.0: 5120,
 16.0: 5120})
df_play['playtime_ratio'] = df_play['playtime'] / df_play['duration_mean']
df_play['isskip'] = (df_play['playtime_ratio'] < 0.05).astype('int8')
df_play['isend'] = (df_play['playtime_ratio'] >0.9).astype('int8')

In [36]:
%%time
df_play = df_play.groupby(['user_id','date']).agg(
{
    'item_id':['count','nunique'],
    'playtime':['max','min','std','mean','median','sum'],
    'duration':['count','max','min','median','nunique'],
    'duration_mean':['max','min','std','mean','sum','median'],
     'playtime_ratio':['max','min','std','mean','median'],
    'father_id_count':['count','mean','max','min'],
    'father_id_freq':['max','mean'],
    'father_id_score':['max','min','mean','std'],
    'item_id_freq':['max','mean'],
    'item_id_score':['max','min','mean','std'],
    'isskip':['sum','mean'],
    'isend':['sum','mean'],
}
)
df_play.reset_index(inplace=True)
df_play.columns = [''.join(col ) for col in df_play.columns]
df_play['playtime_ratio_day_mean'] = df_play['playtimesum'] / df_play['duration_meansum']

Wall time: 6min 17s


In [37]:
df_play.head() 

,user_id,date,item_idcount,item_idnunique,playtimemax,playtimemin,playtimestd,playtimemean,playtimemedian,playtimesum,...,item_id_freqmean,item_id_scoremax,item_id_scoremin,item_id_scoremean,item_id_scorestd,isskipsum,isskipmean,isendsum,isendmean,playtime_ratio_day_mean
0,10000000,131,12,12,2408.826904,11.023000,907.797044,1853.103027,2401.932617,22237.236328,...,64.555556,5.885799,5.217716,5.420889,0.191498,1.0,0.083333,2.0,0.166667,5.714712
1,10000000,132,2,2,1241.108032,8.096000,871.871169,624.601990,624.601990,1249.203979,...,1.000000,5.385799,5.385799,5.385799,NaN,1.0,0.500000,1.0,0.500000,0.481546
2,10000000,164,1,1,557.606018,557.606018,NaN,557.606018,557.606018,557.606018,...,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.0,0.000000,inf
3,10000000,195,2,2,2567.687988,19.482000,1801.853734,1293.584961,1293.584961,2587.169922,...,1.000000,5.885799,3.385799,4.635799,1.767767,1.0,0.500000,0.0,0.000000,1.994618
4,10000001,117,2,2,2519.238037,2512.110107,5.040207,2515.674072,2515.674072,5031.348145,...,7612.000000,3.904475,3.904475,3.904475,NaN,0.0,0.000000,0.0,0.000000,inf


In [38]:
def f(x):
    if len(x)==0 or x == 'nan':
        return np.nan
    if ';' in x:
        return np.mean([int(num) for num in x.split(';')])
    return float(x)
df_port['device_ram'] = df_port['device_ram'].astype('str').apply(f)
df_port['device_rom'] = df_port['device_rom'].astype('str').apply(f)
df_port.territory_code -= 860000
df_port['territory_code_cut'] = df_port.territory_code // 1000

In [39]:
df_port['label'] = df_port['user_id'].map(df[df.label>=0].groupby('user_id')['label'].mean())

In [40]:
territory_score = target_encoding("territory_code", df_port[df_port.label>=0])
del df_port['label']
territory_score 

,territory_code,freq,in_category,weight,score
0,0.0,1,0.000000,0.500000,0.813919
1,1100.0,12128,2.008708,0.999918,2.008676
2,1200.0,6267,1.852582,0.999840,1.852546
3,1301.0,4429,1.810205,0.999774,1.810164
4,1302.0,2728,1.668786,0.999634,1.668771
...,...,...,...,...,...
368,7107.0,13,2.035763,0.928571,2.006626
369,7108.0,36,1.972796,0.972973,1.963473
370,7109.0,10,1.275501,0.909091,1.307532
371,8100.0,112,1.412291,0.991150,1.414199


In [41]:
df_port = df_port.merge(
    territory_score[['territory_code','freq','score']].rename(columns = {
        'freq':'territory_code_freq','score':'territory_code_score'
    }),
    on = 'territory_code',
    how='left'
)

In [42]:
%%time
df = df.merge(df_play,on=['user_id','date'],how='left')
df = df.merge(df_inter,on=['user_id','date'],how='left')
df = df.merge(df_port,on=['user_id'],how='left')
df = df.merge(
    date_score[['date','freq','score']].rename(columns ={
        'freq':'date_freq','score':'date_score'
    }),
    on = 'date',
    how='left'
)
# df = df.merge(df_inter_pivot,on=['user_id','date'],how='left')
# df = df.merge(df_play_pivot,on=['user_id','date'],how='left')

Wall time: 26.2 s


In [43]:
df_inter.columns

Index(['user_id', 'date', 'inter_count', 'inter_item_nunique',
       'inter_type_nunique', 'inter_ratio'],
      dtype='object')

In [44]:
# for col in df_inter_pivot.columns[2:]:
#     df[col] = df[col].fillna(0).astype('int8')
# for col in df_play_pivot.columns[2:]:
#     df[col] = df[col].fillna(0).astype('int8')

In [45]:
from tqdm.notebook import tqdm

In [46]:
@numba.jit(nopython=True,fastmath=True)
def last_n_days_sum(arr,n_days):
    res = np.zeros(len(arr),dtype=np.float32)
    res -= 1
    
    for i in range(0,len(arr)):
        tmp_sum = 0
        for j in range(1,n_days+1):
            if i - j >= 0 and arr[i-j][0] == arr[i][0] and arr[i][1] - arr[i-j][1] <= n_days:
                tmp_sum += arr[i-j][2]
            else:
                break
        res[i] = tmp_sum
    return res


for n_days in tqdm(range(1,8)):
    for col in ['islogin','playtimesum','inter_count','launch_count','type1_count','type0_count',
                'playtime_ratio_day_mean','item_idnunique']:
        df[f'last_{n_days}days_{col}_sum'] = last_n_days_sum(df[['user_id','date',col]].fillna(0).values,n_days)
        df[f'last_{n_days}days_{col}_sum'] = np.where(df['day_minus_min_date'] < n_days,
                                                  np.nan,
                                                 df[f'last_{n_days}days_{col}_sum']) 
        df[f'last_{n_days}days_{col}_sum'] = np.where(df[f'last_{n_days}days_{col}_sum']< 0,
                                                  np.nan,
                                                 df[f'last_{n_days}days_{col}_sum'])
        if n_days == 1:
            df[f'last_{n_days}days_{col}_sum'] = df[f'last_{n_days}days_{col}_sum'].fillna(0)
        else:
            df[f'last_{n_days}days_{col}_sum'] = np.where(df[f'last_{n_days}days_{col}_sum'].isnull(),
                                                         df[f'last_{n_days-1}days_{col}_sum'] *(n/(n-1)),
                                                          df[f'last_{n_days}days_{col}_sum']
                                                         )

In [47]:
df.columns

Index(['user_id', 'date', 'launch_count', 'type1_count', 'type0_count',
       'islogin', 'label', 'future_1_login_mean', 'future_2_login_mean',
       'future_3_login_mean',
       ...
       'last_6days_playtime_ratio_day_mean_sum',
       'last_6days_item_idnunique_sum', 'last_7days_islogin_sum',
       'last_7days_playtimesum_sum', 'last_7days_inter_count_sum',
       'last_7days_launch_count_sum', 'last_7days_type1_count_sum',
       'last_7days_type0_count_sum', 'last_7days_playtime_ratio_day_mean_sum',
       'last_7days_item_idnunique_sum'],
      dtype='object', length=154)

In [48]:
df = reduce_mem(df)

7593.41 Mb, 3440.76 Mb (54.69 %)


In [49]:
@numba.jit(nopython=True)
def ndays_myshift(arr,ndays):
    res = np.zeros(len(arr),dtype=np.float32)
    #res -= 1
  #  login_day = np.zeros(300,dtype=np.int8)
    tmp_id = -1
    for i in range(len(arr)):
        if arr[i][0] != tmp_id:
            login_day = np.zeros(300,dtype=np.float32)
            tmp_id = arr[i][0]
            login_day[int(arr[i][1])] = arr[i][2]
        else:
            login_day[int(arr[i][1])] = arr[i][2]
            res[i] = login_day[int(arr[i][1])-ndays]
                
    return res
for col in tqdm(['playtimesum','item_idcount','inter_count','islogin','launch_count',
                 'type1_count','type0_count','playtime_ratio_day_mean']):
    for ndays in range(1,8):
        df[f'shift_{col}_{ndays}'] = ndays_myshift(df[['user_id','date',col]].fillna(0).values,ndays)
        df[f'shift_{col}_{ndays}'] = df[f'shift_{col}_{ndays}'].fillna(0)
        df[f'shift_{col}_{ndays}'] = np.where(df['day_minus_min_date'] < ndays,
                                                  np.nan,df[f'shift_{col}_{ndays}'])
        df[f'shift_{col}_{ndays}'] = np.where(df[f'shift_{col}_{ndays}']<0,
                                             np.nan,df[f'shift_{col}_{ndays}'])
        

for i in range(3,8):
    df[f'launch_mode{i}'] = 0
    for j in range(1,i+1):
        df[f'launch_mode{i}'] += df[f'shift_islogin_{j}'] * 2 ** (j-1) 
    
# df['weight_launch'] = 0
# df['weight_playtimesum'] = 0
# df['weight_item_idcount'] = 0
# alpha = 0.7
# for i in range(1,8):
#     df['weight_launch'] += df[f'shift_islogin_{i}'] * alpha ** i 
#     df['weight_playtimesum'] += df[f'shift_playtimesum_{i}'] * alpha ** i 
#     df['weight_item_idcount'] += df[f'shift_item_idcount_{i}'] * alpha ** i 

In [50]:
df['weekday'] = df['date'] % 7

In [51]:
df['c1'] = df['sex'].astype('str') + '_' + df['age'].astype('str')

In [52]:
cat_cols =  ['education', 'occupation_status','device_type','sex','weekday',
             'territory_code','territory_code_cut','c1'] + [f'launch_mode{i}' for i in range(3,8)]
for col in cat_cols:
    df[col] = pd.factorize(df[col].fillna(-1))[0]

In [53]:
del df_video
del df_app
del df_port
del df_inter
del df_inter_pivot
del df_play_pivot

In [54]:

df = reduce_mem(df)

6118.79 Mb, 4889.95 Mb (20.08 %)


In [55]:
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics
from sklearn import preprocessing
def myscore(y_label,y_pred):
    score = np.mean(np.abs(y_label-y_pred))
    return score
drop_cols = ['user_id','label','shift_1_label','tmp','date']
x_cols = [col for col in df.columns if col not in drop_cols 
      #    and 'label' not in col
         ]
len(x_cols)

214

In [56]:
# for col in x_cols:
#     if col not in cat_cols:
#         df[col] = (df[col] - df[col].mean()) / df[col].std()
# df = df.fillna(0).replace([float('inf'),-float('inf')],0)
# df.to_feather('df.feather')

In [57]:
import gc
df_train = train_date.merge(df[x_cols+['user_id','date','label']],on=['user_id','date'],how='left')
df.to_feather('df.feather')
#del df
gc.collect()

0

In [58]:
df_tmp = df[df.label==-1]

In [59]:
import warnings
warnings.filterwarnings('ignore')

In [60]:
ss_data = []
for col in x_cols.copy():
    if col not in cat_cols:
        df_train[col] = df_train[col].fillna(0)
        tmp_mean = df_train[col].mean()
        tmp_std = df_train[col].std()
        df_train[col] = (df_train[col] - tmp_mean) / tmp_std
        df_tmp[col] = (df_tmp[col] - tmp_mean) / tmp_std
        if np.isnan(tmp_mean) or np.isnan(tmp_std):
            x_cols.remove(col)
df_train = df_train.fillna(0).replace([float('inf'),-float('inf')],0)
df_tmp = df_tmp.fillna(0).replace([float('inf'),-float('inf')],0)

In [61]:
df_train.index = np.arange(len(df_train))

In [62]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

In [63]:
import torch
import random

In [64]:
def seed_torch(seed=0):
    seed = int(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic =True
seed_torch(0)

开始模型训练

In [65]:
cat_idxs = [x_cols.index(col) for col in cat_cols]
cat_dims = [df_train[col].nunique() for col in cat_cols]

In [66]:
train_x = df_train[x_cols]
train_y = df_train['label']
oof = np.zeros(len(train_x))
seeds = [0]
models = []
scores = []
for seed in seeds:
    
    kfold = KFold(5, random_state = seed, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_y)):
        print('-'*30)
        model = TabNetRegressor(
                    n_d = 16,
                    n_a = 16,
                    n_steps = 3,
                    n_independent = 3,
                    cat_idxs=cat_idxs,
                    cat_dims=cat_dims,
        )
        
       

        model.fit(train_x.loc[tr_inds].values, train_y.loc[tr_inds].values.reshape(-1, 1),
                  patience = 30,
                  eval_set = [
                      (train_x.loc[tr_inds].values, train_y.loc[tr_inds].values.reshape(-1, 1)),
                      (train_x.loc[val_inds].values, train_y.loc[val_inds].values.reshape(-1, 1))
                  ],
                   eval_metric =['mae'],
                  batch_size = 8192,
                  virtual_batch_size = 512,
                  max_epochs = 1000,
#                   categorical_feature = cat_cols,
#                   verbose = 50
                  
                  
                 )
        models.append(model)
        preds = model.predict(train_x.loc[val_inds].values)
        oof[val_inds] += preds.flatten() / len(seeds)
#         score = myscore(train_y.loc[val_inds].values,preds)
#         scores.append(score)
        #print(score)

print(myscore(train_y,oof),np.mean(scores))

------------------------------
Device used : cuda
epoch 0  | loss: 2.79561 | val_0_mae: 1.0758800506591797| val_1_mae: 1.0815999507904053|  0:00:20s
epoch 1  | loss: 1.93658 | val_0_mae: 1.01528000831604| val_1_mae: 1.0210100412368774|  0:00:37s
epoch 2  | loss: 1.84939 | val_0_mae: 1.0032800436019897| val_1_mae: 1.0096900463104248|  0:00:54s
epoch 3  | loss: 1.82765 | val_0_mae: 0.9989200234413147| val_1_mae: 1.0047999620437622|  0:01:11s
epoch 4  | loss: 1.81381 | val_0_mae: 0.9974899888038635| val_1_mae: 1.003600001335144|  0:01:27s
epoch 5  | loss: 1.80565 | val_0_mae: 1.0017499923706055| val_1_mae: 1.0073599815368652|  0:01:44s
epoch 6  | loss: 1.7945  | val_0_mae: 0.9866099953651428| val_1_mae: 0.9935299754142761|  0:02:01s
epoch 7  | loss: 1.78576 | val_0_mae: 0.983709990978241| val_1_mae: 0.9897000193595886|  0:02:17s
epoch 8  | loss: 1.77355 | val_0_mae: 0.9911999702453613| val_1_mae: 0.9979900121688843|  0:02:34s
epoch 9  | loss: 1.76681 | val_0_mae: 0.9827799797058105| val_1

epoch 2  | loss: 1.88655 | val_0_mae: 1.014930009841919| val_1_mae: 1.0110399723052979|  0:00:49s
epoch 3  | loss: 1.83568 | val_0_mae: 1.0073200464248657| val_1_mae: 1.0026600360870361|  0:01:06s
epoch 4  | loss: 1.81318 | val_0_mae: 1.0110700130462646| val_1_mae: 1.006309986114502|  0:01:22s
epoch 5  | loss: 1.80065 | val_0_mae: 0.9941099882125854| val_1_mae: 0.9896199703216553|  0:01:38s
epoch 6  | loss: 1.79001 | val_0_mae: 0.9884099960327148| val_1_mae: 0.9836699962615967|  0:01:55s
epoch 7  | loss: 1.79274 | val_0_mae: 0.9773899912834167| val_1_mae: 0.9726499915122986|  0:02:11s
epoch 8  | loss: 1.79482 | val_0_mae: 0.9925100207328796| val_1_mae: 0.9874100089073181|  0:02:28s
epoch 9  | loss: 1.79221 | val_0_mae: 0.9758300185203552| val_1_mae: 0.9701399803161621|  0:02:44s
epoch 10 | loss: 1.78446 | val_0_mae: 0.984220027923584| val_1_mae: 0.9793000221252441|  0:03:01s
epoch 11 | loss: 1.77528 | val_0_mae: 0.9867900013923645| val_1_mae: 0.9818000197410583|  0:03:17s
epoch 12 | lo

epoch 44 | loss: 1.7215  | val_0_mae: 0.9784799814224243| val_1_mae: 0.9875199794769287|  0:12:22s
epoch 45 | loss: 1.71518 | val_0_mae: 0.9836199879646301| val_1_mae: 0.9922900199890137|  0:12:39s
epoch 46 | loss: 1.71262 | val_0_mae: 0.9691399931907654| val_1_mae: 0.9789599776268005|  0:12:55s
epoch 47 | loss: 1.71151 | val_0_mae: 0.9732099771499634| val_1_mae: 0.9840199947357178|  0:13:12s
epoch 48 | loss: 1.70935 | val_0_mae: 0.9681800007820129| val_1_mae: 0.9795399904251099|  0:13:28s
epoch 49 | loss: 1.70672 | val_0_mae: 0.9809399843215942| val_1_mae: 0.9925500154495239|  0:13:44s
epoch 50 | loss: 1.70763 | val_0_mae: 0.9706799983978271| val_1_mae: 0.9822099804878235|  0:14:01s
epoch 51 | loss: 1.70574 | val_0_mae: 0.9728999733924866| val_1_mae: 0.9851400256156921|  0:14:17s
epoch 52 | loss: 1.70257 | val_0_mae: 0.9755899906158447| val_1_mae: 0.9884099960327148|  0:14:34s
epoch 53 | loss: 1.70341 | val_0_mae: 0.9819899797439575| val_1_mae: 0.9948700070381165|  0:14:50s
epoch 54 |

epoch 60 | loss: 1.70129 | val_0_mae: 0.9633899927139282| val_1_mae: 0.9758300185203552|  0:16:42s
epoch 61 | loss: 1.69816 | val_0_mae: 0.9726799726486206| val_1_mae: 0.984250009059906|  0:16:59s
epoch 62 | loss: 1.70225 | val_0_mae: 0.9861199855804443| val_1_mae: 0.9958299994468689|  0:17:15s
epoch 63 | loss: 1.70523 | val_0_mae: 0.9773600101470947| val_1_mae: 0.9891499876976013|  0:17:32s
epoch 64 | loss: 1.69747 | val_0_mae: 0.9712100028991699| val_1_mae: 0.9835799932479858|  0:17:48s
epoch 65 | loss: 1.69417 | val_0_mae: 0.9894800186157227| val_1_mae: 1.0026799440383911|  0:18:05s
epoch 66 | loss: 1.69257 | val_0_mae: 0.9736700057983398| val_1_mae: 0.9876700043678284|  0:18:21s
epoch 67 | loss: 1.69204 | val_0_mae: 0.9590399861335754| val_1_mae: 0.9732199907302856|  0:18:38s
epoch 68 | loss: 1.6874  | val_0_mae: 0.9660599827766418| val_1_mae: 0.9806900024414062|  0:18:54s
epoch 69 | loss: 1.69035 | val_0_mae: 0.9743199944496155| val_1_mae: 0.988510012626648|  0:19:10s
epoch 70 | l

epoch 53 | loss: 1.70875 | val_0_mae: 0.9638100266456604| val_1_mae: 0.9725900292396545|  0:14:50s
epoch 54 | loss: 1.70843 | val_0_mae: 0.9617400169372559| val_1_mae: 0.96875 |  0:15:07s
epoch 55 | loss: 1.71317 | val_0_mae: 0.9742000102996826| val_1_mae: 0.9826300144195557|  0:15:23s
epoch 56 | loss: 1.70668 | val_0_mae: 0.9660599827766418| val_1_mae: 0.9753400087356567|  0:15:40s
epoch 57 | loss: 1.70772 | val_0_mae: 0.9656199812889099| val_1_mae: 0.9752399921417236|  0:15:57s

Early stopping occurred at epoch 57 with best_epoch = 27 and best_val_1_mae = 0.9684799909591675
Best weights from best epoch are automatically used!
0.967249245113862 nan


In [67]:
# train_x = df_train[x_cols]
# train_y = df_train['label']
# oof = np.zeros(len(train_x))
# seeds = [0]
# models = []
# scores = []
# for seed in seeds:
    
#     kfold = KFold(5, random_state = seed, shuffle = True)
#     for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(train_y)):
#         print('-'*30)
#         model = LGBMRegressor(
#             num_leaves=31,
# #             max_depth = 9,
# #             subsample = 0.9,
#            # colsample_bytree = 0.5,
#             learning_rate=0.1,
#             n_estimators=3000,
#             min_child_samples=20,
#             #reg_lambda  = 3,
#             random_state = seed
#         )
        
       

#         model.fit(train_x.loc[tr_inds], train_y.loc[tr_inds],early_stopping_rounds =100,
#                   eval_set = [(train_x.loc[tr_inds], train_y.loc[tr_inds]),(train_x.loc[val_inds], train_y.loc[val_inds])],
#                    eval_metric ='l1',
#                   categorical_feature = cat_cols,
#                   verbose = 50
                  
                  
#                  )
#         models.append(model)
#         preds = model.predict(train_x.loc[val_inds])
#         oof[val_inds] += preds / len(seeds)
#         score = myscore(train_y.loc[val_inds],preds)
#         scores.append(score)
#         #print(score)

# print(myscore(train_y,oof),np.mean(scores))

In [68]:
print(100*(1-myscore(train_y,oof) / 7))

86.18215364123054


In [72]:
pred = 0
for model in models:
    pred +=  model.predict(df_tmp[x_cols].values).flatten() / len(models)
df_tmp['label'] = pred

抽样偏差处理，详细见思路与方案文档

In [84]:
sub = (test.rename(columns={'end_date':'date'})).merge(df_tmp[['user_id','label','date']],on=['user_id','date'],how='left')
sub['label'] *= 1.05
sub['label'] = round(sub['label'])
sub['label'] = np.where((sub['islogin'] > 0),sub['label'],sub['label'])
sub['label'] = np.where((sub['islogin'] < 0) &(sub['last1_login']>sorted(sub['last1_login'].unique())[24]),
                        0,sub['label'])
sub['label'] = np.where((sub['islogin'] < 0) &(sub['last1_login'].between(sorted(sub['last1_login'].unique())[10],
    sorted(sub['last1_login'].unique())[25]))&(sub['label']>1),
                        1,sub['label'])
sub['label'] = np.where((sub['islogin'] < 0) &(sub['last1_login'].between(sorted(sub['last1_login'].unique())[4],
    sorted(sub['last1_login'].unique())[9]))&(sub['label']>2),
                        2,sub['label'])
sub['label']  = np.where(sub['label'] >7,7,sub['label'] )
sub['label'] = np.where(sub['label'] <0,0,sub['label'] )
sample = pd.read_csv('test_B数据集.csv')
sample['label'] = sub['label']
sample[['user_id','label']].to_csv('final_sub.csv',index=False,header=None,encoding='UTF-8')